In [1]:
!pip install pygeos
import pygeos
import pandas as pd
import nltk
nltk.download('punkt')
import numpy as np
import geopandas as gpd
from shapely import wkt 

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/opt/mamba/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


# Part I : Obtention du dataframe 
## 1.1.Récupération des loyers, des surfaces et de la localisation des résidences

In [2]:
df = pd.read_csv("fr_crous_logement_france_entiere.csv", sep =';')


In [3]:
from functions_for_data_cleaning import *

df = get_loyer(df)
df = get_surface(df)
df = get_localisation(df,df['geocalisation']) 
df

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/onyxia/work/Immobilier-etudiant-/functions_for_data_cleaning.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Loyer"][j] = remove(df["Loyer"][j], 100, 1000)
/home/onyxia/work/Immobilier-etudiant-/functions_for_data_cleaning.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Surface'][j] = remove(df['Surface'][j], 7, 100)


,id,title,short_desc,zone,infos,services,contact,address,mail,phone,...,Max Loyer,Min Loyer,Mean Loyer,Surface,Max Surface,Min Surface,Mean Surface,Longitude,Latitude,geometry
0,205,Résidence Vauban,Guyancourt,YVELINES,Description La résidence est située au cœur de...,Laverie \r\n Salles de travail \r\n Salles d...,Résidence Vauban\r\n 1 boulevard d’Alembert782...,1 boulevard d’Alembert 78280 Guyancourt,residence.vauban@crous-versailles.fr,01 80 73 04 00,...,537.00,333.00,435.000000,[],NaN,NaN,NaN,2.044151,48.779652,POINT (2.04415 48.77965)
1,207,Résidence Le Square de l'Echiquier,Cergy-Pontoise,VAL D'OISE,DescriptionRésidence universitaire nouvellemen...,Wifi \r\n Laverie \r\n salle de sport \r\n ...,Résidence Le Square de l'échiquier\n9 square d...,9 square de l'échiquier 95800 Cergy-Pontoise,residence.lacroix@crous-versailles.fr,01 30 30 38 53,...,334.00,334.00,334.000000,[],NaN,NaN,NaN,2.026887,49.048298,POINT (2.02689 49.04830)
2,214,Résidence Les Bas Longchamps,Bagneux,HAUTS DE SEINE,Description Résidence universitaire située à p...,Laverie \r\n Salles de travail et de détent...,Résidence de Bagneux /Les Bas de Longchamps\r\...,2 rue Abraham Lincoln 92220 Bagneux,residence.bagneux@crous-versailles.fr,01 86 63 00 90,...,359.00,332.00,345.500000,[28.0],28.0,28.0,28.000000,2.316971,48.790691,POINT (2.31697 48.79069)
3,225,Résidence de l’Ile,Bures-sur-Yvette,ESSONNE,Description Résidence universitaire située à p...,"Salles de détente, une avec télévision \r\n ...",Résidence de l’Ile\n14 rue du Docteur Collé - ...,14 rue du Docteur Collé - bâtiment 227 du camp...,residence.ile@crous-versailles.fr,01 69 07 68 96,...,362.00,362.00,362.000000,[18.0],18.0,18.0,18.000000,2.157896,48.698803,POINT (2.15790 48.69880)
4,228,Résidence La Pacaterie,Orsay,ESSONNE,Description Résidence universitaire longée par...,"Salles de détente (télévision, ate...",Résidence La Pacaterie\nBâtiment 499 - rue de ...,Bâtiment 499 - rue de la Pacaterie 91400 Orsay,residence.pacaterie@crous-versailles.fr,01 69 15 75 98,...,154.00,154.00,154.000000,[],NaN,NaN,NaN,2.184678,48.701542,POINT (2.18468 48.70154)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,349,Cité Grémillon (secteur d'Hérouville-Saint-Clair),NaN,Herouville Saint Clair,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,Gratuits (Sauf € = Payants),Cité Grémillon (Secteur d'Hérouville-Saint-Cla...,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,herouville@crous-normandie.fr,02 31 47 61 23,...,556.14,249.85,416.648333,"[9.0, 18.0, 30.0, 49.0, 65.0, 69.0]",69.0,9.0,40.000000,-0.339214,49.203205,POINT (-0.33921 49.20321)
784,331,Résidence Antide Janvier,"Centre ville, sur les quais",Besançon - Centre ville,"Localisation :\r\n Au centre ville, à proximit...",Wifi Inclus dans le loyerPoint restoParking,Résidence Antide Janvier \r\n 49 T1 et de 27 T...,6 rue Antide Janvier 25000 Besançon,pole-hebergement-besancon@crous-besancon.fr,03 81 48 47 50,...,523.00,347.00,457.333333,"[19.0, 35.3]",35.3,19.0,27.150000,6.015343,47.236507,POINT (6.01534 47.23651)
785,335,Résidence Jean Paul Sartre,Centre ville,Belfort,Localisation :\r\n La résidence Sartre est sit...,LaverieParking fermé (15€/mois),"Résidence Jean-Paul Sartre \r\n 130 T1, 11 T1 ...",Rue Gaston Deferre 90000 Belfort,pole-hebergement-nfc@crous-bfc.fr,03.84.21.95.37 - 06.48.41.76.36,...,586.00,280.00,403.571429,"[18.0, 36.0, 51.5]",51.5,18.0,35.166667,6.853714,47.638988,POINT (6.85371 47.63899)
786,416,Maret,Logements proposés : chambres traditionnelles,DIJON CENTRE,NaN,"La Résidence universitaire Maret, située en pl...",Résidence MARET \r\n 3 rue du Docteur Maret \r...,3 rue du Docteur Maret 21000 Dijon,NaN,03.45.34.84.57,...,NaN,NaN,NaN,None,NaN,NaN,NaN,5.035332,47.322498,POINT (5.03533 47.32250)


## 1.2.Récupération des loyers privés des communes (avec la géolocalisation des communes)

In [4]:
df_private = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/8fac6fb7-cd07-4747-8e0b-b101c476f0da", encoding= 'unicode_escape', sep = ";")


On ne garde que le code INSEE, le nom de la ville et le loyer supérieur par m^2


In [5]:
df_private = df_private[["INSEE", "LIBGEO","upr.IPm2"]]

On renomme la colonne INSEE en codgeo pour pouvoir la merger avec la carte des contours des villes.

In [6]:
df_private.rename(columns = {'INSEE':'codgeo'}, inplace = True)

In [7]:
"""
On rajoute un 0 devant le code INSEE quand il ne fait que quatre chiffres (pour uniformiser avec l'autre base). 
"""
for i in range(0,df_private.shape[0]):
    if  len(df_private["codgeo"][i]) == 4: 
            df_private["codgeo"][i] = "0" +  df_private["codgeo"][i]

In [8]:
df_private = df_private.set_index('codgeo') #on réindex le dataframe avec le code INSEE pour merger après. 

On récuppère les contours géographique des communes.

In [9]:
gdf_communes_boundaries = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/e9391593-fa95-4153-aabe-87ca84d197e9")

ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


On ne garde que le code INSEE et les données polygoniales. 

In [10]:
gdf_communes_boundaries.rename(columns = {'INSEE_COM':'codgeo'}, inplace = True)
gdf_communes_boundaries = gdf_communes_boundaries[["codgeo", "geometry"]]
gdf_communes_boundaries = gdf_communes_boundaries.set_index('codgeo')


On concatène les deux datafarmes

In [11]:
gdf_private = pd.concat([df_private,gdf_communes_boundaries], axis = 1, join = "inner")

In [12]:
gdf_private

,LIBGEO,upr.IPm2,geometry
codgeo,,,
01001,L'Abergement-Clémenciat,"11,85487972","POLYGON ((4.95825 46.15334, 4.95810 46.15342, ..."
01002,L'Abergement-de-Varey,"11,09535764","POLYGON ((5.43369 45.98393, 5.43441 45.98428, ..."
01004,Ambérieu-en-Bugey,"12,90422511","POLYGON ((5.40942 45.94592, 5.40949 45.94718, ..."
01005,Ambérieux-en-Dombes,"11,85487972","POLYGON ((4.94292 45.97974, 4.94275 45.97998, ..."
01006,Ambléon,"11,3222872","POLYGON ((5.57088 45.75316, 5.57187 45.75200, ..."
...,...,...,...
97420,Sainte-Suzanne,"12,95850521","POLYGON ((55.57407 -20.99343, 55.57408 -20.993..."
97421,Salazie,"12,62690364","POLYGON ((55.54066 -21.00411, 55.54064 -21.004..."
97422,Le Tampon,"13,45103918","POLYGON ((55.66050 -21.20333, 55.65868 -21.201..."


In [13]:
df

,id,title,short_desc,zone,infos,services,contact,address,mail,phone,...,Max Loyer,Min Loyer,Mean Loyer,Surface,Max Surface,Min Surface,Mean Surface,Longitude,Latitude,geometry
0,205,Résidence Vauban,Guyancourt,YVELINES,Description La résidence est située au cœur de...,Laverie \r\n Salles de travail \r\n Salles d...,Résidence Vauban\r\n 1 boulevard d’Alembert782...,1 boulevard d’Alembert 78280 Guyancourt,residence.vauban@crous-versailles.fr,01 80 73 04 00,...,537.00,333.00,435.000000,[],NaN,NaN,NaN,2.044151,48.779652,POINT (2.04415 48.77965)
1,207,Résidence Le Square de l'Echiquier,Cergy-Pontoise,VAL D'OISE,DescriptionRésidence universitaire nouvellemen...,Wifi \r\n Laverie \r\n salle de sport \r\n ...,Résidence Le Square de l'échiquier\n9 square d...,9 square de l'échiquier 95800 Cergy-Pontoise,residence.lacroix@crous-versailles.fr,01 30 30 38 53,...,334.00,334.00,334.000000,[],NaN,NaN,NaN,2.026887,49.048298,POINT (2.02689 49.04830)
2,214,Résidence Les Bas Longchamps,Bagneux,HAUTS DE SEINE,Description Résidence universitaire située à p...,Laverie \r\n Salles de travail et de détent...,Résidence de Bagneux /Les Bas de Longchamps\r\...,2 rue Abraham Lincoln 92220 Bagneux,residence.bagneux@crous-versailles.fr,01 86 63 00 90,...,359.00,332.00,345.500000,[28.0],28.0,28.0,28.000000,2.316971,48.790691,POINT (2.31697 48.79069)
3,225,Résidence de l’Ile,Bures-sur-Yvette,ESSONNE,Description Résidence universitaire située à p...,"Salles de détente, une avec télévision \r\n ...",Résidence de l’Ile\n14 rue du Docteur Collé - ...,14 rue du Docteur Collé - bâtiment 227 du camp...,residence.ile@crous-versailles.fr,01 69 07 68 96,...,362.00,362.00,362.000000,[18.0],18.0,18.0,18.000000,2.157896,48.698803,POINT (2.15790 48.69880)
4,228,Résidence La Pacaterie,Orsay,ESSONNE,Description Résidence universitaire longée par...,"Salles de détente (télévision, ate...",Résidence La Pacaterie\nBâtiment 499 - rue de ...,Bâtiment 499 - rue de la Pacaterie 91400 Orsay,residence.pacaterie@crous-versailles.fr,01 69 15 75 98,...,154.00,154.00,154.000000,[],NaN,NaN,NaN,2.184678,48.701542,POINT (2.18468 48.70154)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,349,Cité Grémillon (secteur d'Hérouville-Saint-Clair),NaN,Herouville Saint Clair,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,Gratuits (Sauf € = Payants),Cité Grémillon (Secteur d'Hérouville-Saint-Cla...,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,herouville@crous-normandie.fr,02 31 47 61 23,...,556.14,249.85,416.648333,"[9.0, 18.0, 30.0, 49.0, 65.0, 69.0]",69.0,9.0,40.000000,-0.339214,49.203205,POINT (-0.33921 49.20321)
784,331,Résidence Antide Janvier,"Centre ville, sur les quais",Besançon - Centre ville,"Localisation :\r\n Au centre ville, à proximit...",Wifi Inclus dans le loyerPoint restoParking,Résidence Antide Janvier \r\n 49 T1 et de 27 T...,6 rue Antide Janvier 25000 Besançon,pole-hebergement-besancon@crous-besancon.fr,03 81 48 47 50,...,523.00,347.00,457.333333,"[19.0, 35.3]",35.3,19.0,27.150000,6.015343,47.236507,POINT (6.01534 47.23651)
785,335,Résidence Jean Paul Sartre,Centre ville,Belfort,Localisation :\r\n La résidence Sartre est sit...,LaverieParking fermé (15€/mois),"Résidence Jean-Paul Sartre \r\n 130 T1, 11 T1 ...",Rue Gaston Deferre 90000 Belfort,pole-hebergement-nfc@crous-bfc.fr,03.84.21.95.37 - 06.48.41.76.36,...,586.00,280.00,403.571429,"[18.0, 36.0, 51.5]",51.5,18.0,35.166667,6.853714,47.638988,POINT (6.85371 47.63899)
786,416,Maret,Logements proposés : chambres traditionnelles,DIJON CENTRE,NaN,"La Résidence universitaire Maret, située en pl...",Résidence MARET \r\n 3 rue du Docteur Maret \r...,3 rue du Docteur Maret 21000 Dijon,NaN,03.45.34.84.57,...,NaN,NaN,NaN,None,NaN,NaN,NaN,5.035332,47.322498,POINT (5.03533 47.32250)


## 1.3. Jointures spatiales

In [14]:
df

,id,title,short_desc,zone,infos,services,contact,address,mail,phone,...,Max Loyer,Min Loyer,Mean Loyer,Surface,Max Surface,Min Surface,Mean Surface,Longitude,Latitude,geometry
0,205,Résidence Vauban,Guyancourt,YVELINES,Description La résidence est située au cœur de...,Laverie \r\n Salles de travail \r\n Salles d...,Résidence Vauban\r\n 1 boulevard d’Alembert782...,1 boulevard d’Alembert 78280 Guyancourt,residence.vauban@crous-versailles.fr,01 80 73 04 00,...,537.00,333.00,435.000000,[],NaN,NaN,NaN,2.044151,48.779652,POINT (2.04415 48.77965)
1,207,Résidence Le Square de l'Echiquier,Cergy-Pontoise,VAL D'OISE,DescriptionRésidence universitaire nouvellemen...,Wifi \r\n Laverie \r\n salle de sport \r\n ...,Résidence Le Square de l'échiquier\n9 square d...,9 square de l'échiquier 95800 Cergy-Pontoise,residence.lacroix@crous-versailles.fr,01 30 30 38 53,...,334.00,334.00,334.000000,[],NaN,NaN,NaN,2.026887,49.048298,POINT (2.02689 49.04830)
2,214,Résidence Les Bas Longchamps,Bagneux,HAUTS DE SEINE,Description Résidence universitaire située à p...,Laverie \r\n Salles de travail et de détent...,Résidence de Bagneux /Les Bas de Longchamps\r\...,2 rue Abraham Lincoln 92220 Bagneux,residence.bagneux@crous-versailles.fr,01 86 63 00 90,...,359.00,332.00,345.500000,[28.0],28.0,28.0,28.000000,2.316971,48.790691,POINT (2.31697 48.79069)
3,225,Résidence de l’Ile,Bures-sur-Yvette,ESSONNE,Description Résidence universitaire située à p...,"Salles de détente, une avec télévision \r\n ...",Résidence de l’Ile\n14 rue du Docteur Collé - ...,14 rue du Docteur Collé - bâtiment 227 du camp...,residence.ile@crous-versailles.fr,01 69 07 68 96,...,362.00,362.00,362.000000,[18.0],18.0,18.0,18.000000,2.157896,48.698803,POINT (2.15790 48.69880)
4,228,Résidence La Pacaterie,Orsay,ESSONNE,Description Résidence universitaire longée par...,"Salles de détente (télévision, ate...",Résidence La Pacaterie\nBâtiment 499 - rue de ...,Bâtiment 499 - rue de la Pacaterie 91400 Orsay,residence.pacaterie@crous-versailles.fr,01 69 15 75 98,...,154.00,154.00,154.000000,[],NaN,NaN,NaN,2.184678,48.701542,POINT (2.18468 48.70154)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,349,Cité Grémillon (secteur d'Hérouville-Saint-Clair),NaN,Herouville Saint Clair,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,Gratuits (Sauf € = Payants),Cité Grémillon (Secteur d'Hérouville-Saint-Cla...,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,herouville@crous-normandie.fr,02 31 47 61 23,...,556.14,249.85,416.648333,"[9.0, 18.0, 30.0, 49.0, 65.0, 69.0]",69.0,9.0,40.000000,-0.339214,49.203205,POINT (-0.33921 49.20321)
784,331,Résidence Antide Janvier,"Centre ville, sur les quais",Besançon - Centre ville,"Localisation :\r\n Au centre ville, à proximit...",Wifi Inclus dans le loyerPoint restoParking,Résidence Antide Janvier \r\n 49 T1 et de 27 T...,6 rue Antide Janvier 25000 Besançon,pole-hebergement-besancon@crous-besancon.fr,03 81 48 47 50,...,523.00,347.00,457.333333,"[19.0, 35.3]",35.3,19.0,27.150000,6.015343,47.236507,POINT (6.01534 47.23651)
785,335,Résidence Jean Paul Sartre,Centre ville,Belfort,Localisation :\r\n La résidence Sartre est sit...,LaverieParking fermé (15€/mois),"Résidence Jean-Paul Sartre \r\n 130 T1, 11 T1 ...",Rue Gaston Deferre 90000 Belfort,pole-hebergement-nfc@crous-bfc.fr,03.84.21.95.37 - 06.48.41.76.36,...,586.00,280.00,403.571429,"[18.0, 36.0, 51.5]",51.5,18.0,35.166667,6.853714,47.638988,POINT (6.85371 47.63899)
786,416,Maret,Logements proposés : chambres traditionnelles,DIJON CENTRE,NaN,"La Résidence universitaire Maret, située en pl...",Résidence MARET \r\n 3 rue du Docteur Maret \r...,3 rue du Docteur Maret 21000 Dijon,NaN,03.45.34.84.57,...,NaN,NaN,NaN,None,NaN,NaN,NaN,5.035332,47.322498,POINT (5.03533 47.32250)


In [15]:
gdf_private

,LIBGEO,upr.IPm2,geometry
codgeo,,,
01001,L'Abergement-Clémenciat,"11,85487972","POLYGON ((4.95825 46.15334, 4.95810 46.15342, ..."
01002,L'Abergement-de-Varey,"11,09535764","POLYGON ((5.43369 45.98393, 5.43441 45.98428, ..."
01004,Ambérieu-en-Bugey,"12,90422511","POLYGON ((5.40942 45.94592, 5.40949 45.94718, ..."
01005,Ambérieux-en-Dombes,"11,85487972","POLYGON ((4.94292 45.97974, 4.94275 45.97998, ..."
01006,Ambléon,"11,3222872","POLYGON ((5.57088 45.75316, 5.57187 45.75200, ..."
...,...,...,...
97420,Sainte-Suzanne,"12,95850521","POLYGON ((55.57407 -20.99343, 55.57408 -20.993..."
97421,Salazie,"12,62690364","POLYGON ((55.54066 -21.00411, 55.54064 -21.004..."
97422,Le Tampon,"13,45103918","POLYGON ((55.66050 -21.20333, 55.65868 -21.201..."


In [16]:

#gdf_private['geometry'] = gdf_private['geometry'].apply(wkt.loads) #on transforme en format wkt (sinon ça ne marche pas)
gdf_private = gpd.GeoDataFrame(gdf_private)  #on transforme en geodataframe 
df = gpd.sjoin(df, gdf_private, how='left', op='within') #on réalise la jointure à gauche avec within 
#i.e si le point est dans le polynôme. 

/opt/mamba/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3373: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_1559/2580680376.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  df = gpd.sjoin(df, gdf_private, how='left', op='within') #on réalise la jointure à gauche avec within


In [17]:
df

,id,title,short_desc,zone,infos,services,contact,address,mail,phone,...,Surface,Max Surface,Min Surface,Mean Surface,Longitude,Latitude,geometry,index_right,LIBGEO,upr.IPm2
0,205,Résidence Vauban,Guyancourt,YVELINES,Description La résidence est située au cœur de...,Laverie \r\n Salles de travail \r\n Salles d...,Résidence Vauban\r\n 1 boulevard d’Alembert782...,1 boulevard d’Alembert 78280 Guyancourt,residence.vauban@crous-versailles.fr,01 80 73 04 00,...,[],NaN,NaN,NaN,2.044151,48.779652,POINT (2.04415 48.77965),78297,Guyancourt,"21,73842264"
1,207,Résidence Le Square de l'Echiquier,Cergy-Pontoise,VAL D'OISE,DescriptionRésidence universitaire nouvellemen...,Wifi \r\n Laverie \r\n salle de sport \r\n ...,Résidence Le Square de l'échiquier\n9 square d...,9 square de l'échiquier 95800 Cergy-Pontoise,residence.lacroix@crous-versailles.fr,01 30 30 38 53,...,[],NaN,NaN,NaN,2.026887,49.048298,POINT (2.02689 49.04830),95127,Cergy,"20,65371054"
2,214,Résidence Les Bas Longchamps,Bagneux,HAUTS DE SEINE,Description Résidence universitaire située à p...,Laverie \r\n Salles de travail et de détent...,Résidence de Bagneux /Les Bas de Longchamps\r\...,2 rue Abraham Lincoln 92220 Bagneux,residence.bagneux@crous-versailles.fr,01 86 63 00 90,...,[28.0],28.0,28.0,28.000000,2.316971,48.790691,POINT (2.31697 48.79069),92007,Bagneux,"24,98281618"
3,225,Résidence de l’Ile,Bures-sur-Yvette,ESSONNE,Description Résidence universitaire située à p...,"Salles de détente, une avec télévision \r\n ...",Résidence de l’Ile\n14 rue du Docteur Collé - ...,14 rue du Docteur Collé - bâtiment 227 du camp...,residence.ile@crous-versailles.fr,01 69 07 68 96,...,[18.0],18.0,18.0,18.000000,2.157896,48.698803,POINT (2.15790 48.69880),91122,Bures-sur-Yvette,"21,2713506"
4,228,Résidence La Pacaterie,Orsay,ESSONNE,Description Résidence universitaire longée par...,"Salles de détente (télévision, ate...",Résidence La Pacaterie\nBâtiment 499 - rue de ...,Bâtiment 499 - rue de la Pacaterie 91400 Orsay,residence.pacaterie@crous-versailles.fr,01 69 15 75 98,...,[],NaN,NaN,NaN,2.184678,48.701542,POINT (2.18468 48.70154),91471,Orsay,"21,07028697"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,349,Cité Grémillon (secteur d'Hérouville-Saint-Clair),NaN,Herouville Saint Clair,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,Gratuits (Sauf € = Payants),Cité Grémillon (Secteur d'Hérouville-Saint-Cla...,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,herouville@crous-normandie.fr,02 31 47 61 23,...,"[9.0, 18.0, 30.0, 49.0, 65.0, 69.0]",69.0,9.0,40.000000,-0.339214,49.203205,POINT (-0.33921 49.20321),14327,Hérouville-Saint-Clair,"13,70540711"
784,331,Résidence Antide Janvier,"Centre ville, sur les quais",Besançon - Centre ville,"Localisation :\r\n Au centre ville, à proximit...",Wifi Inclus dans le loyerPoint restoParking,Résidence Antide Janvier \r\n 49 T1 et de 27 T...,6 rue Antide Janvier 25000 Besançon,pole-hebergement-besancon@crous-besancon.fr,03 81 48 47 50,...,"[19.0, 35.3]",35.3,19.0,27.150000,6.015343,47.236507,POINT (6.01534 47.23651),25056,Besançon,"13,92609502"
785,335,Résidence Jean Paul Sartre,Centre ville,Belfort,Localisation :\r\n La résidence Sartre est sit...,LaverieParking fermé (15€/mois),"Résidence Jean-Paul Sartre \r\n 130 T1, 11 T1 ...",Rue Gaston Deferre 90000 Belfort,pole-hebergement-nfc@crous-bfc.fr,03.84.21.95.37 - 06.48.41.76.36,...,"[18.0, 36.0, 51.5]",51.5,18.0,35.166667,6.853714,47.638988,POINT (6.85371 47.63899),90010,Belfort,"13,04684054"
786,416,Maret,Logements proposés : chambres traditionnelles,DIJON CENTRE,NaN,"La Résidence universitaire Maret, située en pl...",Résidence MARET \r\n 3 rue du Docteur Maret \r...,3 rue du Docteur Maret 21000 Dijon,NaN,03.45.34.84.57,...,None,NaN,NaN,NaN,5.035332,47.322498,POINT (5.03533 47.32250),21231,Dijon,"14,38634283"


## 1.4. Association de résidences à une école - Essais

In [18]:
df_schools = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/8f040d23-a09f-4dee-aa19-0892602490d8", sep = ';')
df_schools

,etablissement_id_paysage,uo_lib,nom_court,sigle,type_d_etablissement,typologie_d_universites_et_assimiles,secteur_d_etablissement,vague_contractuelle,localisation,url,...,identifiant_programmes_lolf,statut_operateur_lolf,libelle_missions_lolf,libelle_programmes_lolf,libelle_programme_lolf_chef_de_file,identifiants_autres_programmes_lolf,libelles_autres_programmes_lolf,identifiant_programme_lolf_chef_de_file,libelle_mission_chef_de_file,compte_googleplus
0,6a498,Casa de Velázquez de Madrid,NaN,NaN,Autre établissement,NaN,Public,NaN,Étranger,http://www.casadevelazquez.org/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5YWUA,École supérieure des sciences économiques et c...,NaN,Essec,École,NaN,Privé,Vague E,Île-de-France>Versailles>Val-d'Oise>Cergy,https://www.essec.edu/fr/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0347i,École supérieure d'ingénieurs des travaux de l...,ESITC de Metz,ESITC Metz,École,NaN,Privé,Vague C,Grand Est>Nancy-Metz>Moselle>Metz,http://www.esitc-metz.com/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ylVkq,École supérieure d'ingénieurs Léonard-de-Vinci,ESILV,NaN,École,NaN,Privé,Vague E,Île-de-France>Versailles>Hauts-de-Seine>Courbe...,https://www.esilv.fr/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,etBz7,Université Claude Bernard - Lyon 1,NaN,NaN,Université,Université scientifique et/ou médicale,Public,Vague A,Auvergne-Rhône-Alpes>Lyon>Rhône>Villeurbanne,https://www.univ-lyon1.fr/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,n2vXC,Institut supérieur d’agriculture Rhône-Alpes,Isara-Lyon,NaN,École,NaN,Privé,Vague A,Auvergne-Rhône-Alpes>Lyon>Rhône>Lyon 7e,https://www.isara.fr/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,atbEK,La Rochelle Université,NaN,NaN,Université,Université pluridisciplinaire hors santé,Public,Vague B,Nouvelle-Aquitaine>Poitiers>Charente-Maritime>...,https://www.univ-larochelle.fr/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,06SE7,Université de Bretagne Occidentale,NaN,UBO,Université,Université pluridisciplinaire avec santé,Public,Vague B,Bretagne>Rennes>Finistère>Brest,http://www.univ-brest.fr/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,t6Cq5,Université de Lorraine,NaN,NaN,Grand établissement,NaN,Public,Vague C,Grand Est>Nancy-Metz>Meurthe-et-Moselle>Nancy,http://www.univ-lorraine.fr/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_schools = df_schools.dropna(subset=['coordonnees'])
df_schools = df_schools.reset_index()


In [20]:
get_localisation(df_schools,df_schools['coordonnees'])

,index,etablissement_id_paysage,uo_lib,nom_court,sigle,type_d_etablissement,typologie_d_universites_et_assimiles,secteur_d_etablissement,vague_contractuelle,localisation,...,libelle_programmes_lolf,libelle_programme_lolf_chef_de_file,identifiants_autres_programmes_lolf,libelles_autres_programmes_lolf,identifiant_programme_lolf_chef_de_file,libelle_mission_chef_de_file,compte_googleplus,Longitude,Latitude,geometry
0,0,6a498,Casa de Velázquez de Madrid,NaN,NaN,Autre établissement,NaN,Public,NaN,Étranger,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.730450,40.441380,POINT (-3.73045 40.44138)
1,1,5YWUA,École supérieure des sciences économiques et c...,NaN,Essec,École,NaN,Privé,Vague E,Île-de-France>Versailles>Val-d'Oise>Cergy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.078427,49.033970,POINT (2.07843 49.03397)
2,2,0347i,École supérieure d'ingénieurs des travaux de l...,ESITC de Metz,ESITC Metz,École,NaN,Privé,Vague C,Grand Est>Nancy-Metz>Moselle>Metz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.216160,49.100420,POINT (6.21616 49.10042)
3,3,ylVkq,École supérieure d'ingénieurs Léonard-de-Vinci,ESILV,NaN,École,NaN,Privé,Vague E,Île-de-France>Versailles>Hauts-de-Seine>Courbe...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.235900,48.896200,POINT (2.23590 48.89620)
4,4,etBz7,Université Claude Bernard - Lyon 1,NaN,NaN,Université,Université scientifique et/ou médicale,Public,Vague A,Auvergne-Rhône-Alpes>Lyon>Rhône>Villeurbanne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.865180,45.779620,POINT (4.86518 45.77962)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,245,n2vXC,Institut supérieur d’agriculture Rhône-Alpes,Isara-Lyon,NaN,École,NaN,Privé,Vague A,Auvergne-Rhône-Alpes>Lyon>Rhône>Lyon 7e,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.836377,45.727895,POINT (4.83638 45.72789)
245,246,atbEK,La Rochelle Université,NaN,NaN,Université,Université pluridisciplinaire hors santé,Public,Vague B,Nouvelle-Aquitaine>Poitiers>Charente-Maritime>...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.155040,46.147740,POINT (-1.15504 46.14774)
246,247,06SE7,Université de Bretagne Occidentale,NaN,UBO,Université,Université pluridisciplinaire avec santé,Public,Vague B,Bretagne>Rennes>Finistère>Brest,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.507700,48.398030,POINT (-4.50770 48.39803)
247,248,t6Cq5,Université de Lorraine,NaN,NaN,Grand établissement,NaN,Public,Vague C,Grand Est>Nancy-Metz>Meurthe-et-Moselle>Nancy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.176340,48.696200,POINT (6.17634 48.69620)


In [21]:
import matplotlib.pyplot as plt
a =(df.geometry.buffer(10))
gdf = df
gdf['geometry']= a
#del gdf['id']

In [44]:
#on échange les places de la colonne geometry et de la dernière colonne du dataframe upr.IPm2. Ainsi, la dernière colonne du dataframe contient les coordonnées.
print(gdf.columns)
b = gdf['upr.IPm2']
gdf['upr.IPm2']=gdf['geometry']
gdf['geometry']= b

gdf = gdf.rename(columns={'upr.IPm2':'geometry', 'geometry':'upr.IPm2'})
gdf


Index(['id', 'title', 'short_desc', 'zone', 'infos', 'services', 'contact',
       'address', 'mail', 'phone', 'openinghours', 'interneturl',
       'appointmenturl', 'virtualvisiturl', 'bookingurl', 'troubleshootingurl',
       'geocalisation', 'photo', 'regions', 'Loyer', 'Max Loyer', 'Min Loyer',
       'Mean Loyer', 'Surface', 'Max Surface', 'Min Surface', 'Mean Surface',
       'Longitude', 'Latitude', 'geometry', 'index_right', 'LIBGEO',
       'upr.IPm2'],
      dtype='object')


/opt/mamba/lib/python3.10/site-packages/geopandas/geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,id,title,short_desc,zone,infos,services,contact,address,mail,phone,...,Surface,Max Surface,Min Surface,Mean Surface,Longitude,Latitude,upr.IPm2,index_right,LIBGEO,geometry
0,205,Résidence Vauban,Guyancourt,YVELINES,Description La résidence est située au cœur de...,Laverie \r\n Salles de travail \r\n Salles d...,Résidence Vauban\r\n 1 boulevard d’Alembert782...,1 boulevard d’Alembert 78280 Guyancourt,residence.vauban@crous-versailles.fr,01 80 73 04 00,...,[],NaN,NaN,NaN,2.044151,48.779652,"21,73842264",78297,Guyancourt,"POLYGON ((12.04415 48.77965, 11.99600 47.79948..."
1,207,Résidence Le Square de l'Echiquier,Cergy-Pontoise,VAL D'OISE,DescriptionRésidence universitaire nouvellemen...,Wifi \r\n Laverie \r\n salle de sport \r\n ...,Résidence Le Square de l'échiquier\n9 square d...,9 square de l'échiquier 95800 Cergy-Pontoise,residence.lacroix@crous-versailles.fr,01 30 30 38 53,...,[],NaN,NaN,NaN,2.026887,49.048298,"20,65371054",95127,Cergy,"POLYGON ((12.02689 49.04830, 11.97873 48.06813..."
2,214,Résidence Les Bas Longchamps,Bagneux,HAUTS DE SEINE,Description Résidence universitaire située à p...,Laverie \r\n Salles de travail et de détent...,Résidence de Bagneux /Les Bas de Longchamps\r\...,2 rue Abraham Lincoln 92220 Bagneux,residence.bagneux@crous-versailles.fr,01 86 63 00 90,...,[28.0],28.0,28.0,28.000000,2.316971,48.790691,"24,98281618",92007,Bagneux,"POLYGON ((12.31697 48.79069, 12.26882 47.81052..."
3,225,Résidence de l’Ile,Bures-sur-Yvette,ESSONNE,Description Résidence universitaire située à p...,"Salles de détente, une avec télévision \r\n ...",Résidence de l’Ile\n14 rue du Docteur Collé - ...,14 rue du Docteur Collé - bâtiment 227 du camp...,residence.ile@crous-versailles.fr,01 69 07 68 96,...,[18.0],18.0,18.0,18.000000,2.157896,48.698803,"21,2713506",91122,Bures-sur-Yvette,"POLYGON ((12.15790 48.69880, 12.10974 47.71863..."
4,228,Résidence La Pacaterie,Orsay,ESSONNE,Description Résidence universitaire longée par...,"Salles de détente (télévision, ate...",Résidence La Pacaterie\nBâtiment 499 - rue de ...,Bâtiment 499 - rue de la Pacaterie 91400 Orsay,residence.pacaterie@crous-versailles.fr,01 69 15 75 98,...,[],NaN,NaN,NaN,2.184678,48.701542,"21,07028697",91471,Orsay,"POLYGON ((12.18468 48.70154, 12.13652 47.72137..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,349,Cité Grémillon (secteur d'Hérouville-Saint-Clair),NaN,Herouville Saint Clair,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,Gratuits (Sauf € = Payants),Cité Grémillon (Secteur d'Hérouville-Saint-Cla...,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,herouville@crous-normandie.fr,02 31 47 61 23,...,"[9.0, 18.0, 30.0, 49.0, 65.0, 69.0]",69.0,9.0,40.000000,-0.339214,49.203205,"13,70540711",14327,Hérouville-Saint-Clair,"POLYGON ((9.66079 49.20321, 9.61263 48.22303, ..."
784,331,Résidence Antide Janvier,"Centre ville, sur les quais",Besançon - Centre ville,"Localisation :\r\n Au centre ville, à proximit...",Wifi Inclus dans le loyerPoint restoParking,Résidence Antide Janvier \r\n 49 T1 et de 27 T...,6 rue Antide Janvier 25000 Besançon,pole-hebergement-besancon@crous-besancon.fr,03 81 48 47 50,...,"[19.0, 35.3]",35.3,19.0,27.150000,6.015343,47.236507,"13,92609502",25056,Besançon,"POLYGON ((16.01534 47.23651, 15.96719 46.25634..."
785,335,Résidence Jean Paul Sartre,Centre ville,Belfort,Localisation :\r\n La résidence Sartre est sit...,LaverieParking fermé (15€/mois),"Résidence Jean-Paul Sartre \r\n 130 T1, 11 T1 ...",Rue Gaston Deferre 90000 Belfort,pole-hebergement-nfc@crous-bfc.fr,03.84.21.95.37 - 06.48.41.76.36,...,"[18.0, 36.0, 51.5]",51.5,18.0,35.166667,6.853714,47.638988,"13,04684054",90010,Belfort,"POLYGON ((16.85371 47.63899, 16.80556 46.65882..."
786,416,Maret,Logements proposés : chambres traditionnelles,DIJON CENTRE,NaN,"La Résidence universitaire Maret, située en pl...",Résidence MARET \r\n 3 rue du Docteur Maret \r...,3 rue du Docteur Maret 21000 Dijon,NaN,03.45.34.84.57,...,None,NaN,NaN,NaN,5.03533

In [45]:
gdf = gdf.drop(['index_right'], axis=1)

In [46]:
print(gdf.columns)
gdf2 = gdf[['title','Longitude','Latitude','LIBGEO','geometry']]
gdf2

Index(['id', 'title', 'short_desc', 'zone', 'infos', 'services', 'contact',
       'address', 'mail', 'phone', 'openinghours', 'interneturl',
       'appointmenturl', 'virtualvisiturl', 'bookingurl', 'troubleshootingurl',
       'geocalisation', 'photo', 'regions', 'Loyer', 'Max Loyer', 'Min Loyer',
       'Mean Loyer', 'Surface', 'Max Surface', 'Min Surface', 'Mean Surface',
       'Longitude', 'Latitude', 'upr.IPm2', 'LIBGEO', 'geometry'],
      dtype='object')


,title,Longitude,Latitude,LIBGEO,geometry
0,Résidence Vauban,2.044151,48.779652,Guyancourt,"POLYGON ((12.04415 48.77965, 11.99600 47.79948..."
1,Résidence Le Square de l'Echiquier,2.026887,49.048298,Cergy,"POLYGON ((12.02689 49.04830, 11.97873 48.06813..."
2,Résidence Les Bas Longchamps,2.316971,48.790691,Bagneux,"POLYGON ((12.31697 48.79069, 12.26882 47.81052..."
3,Résidence de l’Ile,2.157896,48.698803,Bures-sur-Yvette,"POLYGON ((12.15790 48.69880, 12.10974 47.71863..."
4,Résidence La Pacaterie,2.184678,48.701542,Orsay,"POLYGON ((12.18468 48.70154, 12.13652 47.72137..."
...,...,...,...,...,...
783,Cité Grémillon (secteur d'Hérouville-Saint-Clair),-0.339214,49.203205,Hérouville-Saint-Clair,"POLYGON ((9.66079 49.20321, 9.61263 48.22303, ..."
784,Résidence Antide Janvier,6.015343,47.236507,Besançon,"POLYGON ((16.01534 47.23651, 15.96719 46.25634..."
785,Résidence Jean Paul Sartre,6.853714,47.638988,Belfort,"POLYGON ((16.85371 47.63899, 16.80556 46.65882..."
786,Maret,5.035332,47.322498,Dijon,"POLYGON ((15.03533 47.32250, 14.98718 46.34233..."


In [47]:
df_schools = df_schools[["geometry","uo_lib","type_d_etablissement","coordonnees","inscrits_2017"]]
df_schools


,geometry,uo_lib,type_d_etablissement,coordonnees,inscrits_2017
0,POINT (-3.73045 40.44138),Casa de Velázquez de Madrid,Autre établissement,"40.44138,-3.73045",nd
1,POINT (2.07843 49.03397),École supérieure des sciences économiques et c...,École,"49.03397,2.078427",6 103
2,POINT (6.21616 49.10042),École supérieure d'ingénieurs des travaux de l...,École,"49.10042,6.21616",237
3,POINT (2.23590 48.89620),École supérieure d'ingénieurs Léonard-de-Vinci,École,"48.8962,2.2359",2 202
4,POINT (4.86518 45.77962),Université Claude Bernard - Lyon 1,Université,"45.77962,4.86518",38 002
...,...,...,...,...,...
244,POINT (4.83638 45.72789),Institut supérieur d’agriculture Rhône-Alpes,École,"45.727895,4.836377",937
245,POINT (-1.15504 46.14774),La Rochelle Université,Université,"46.14774,-1.15504",8 373
246,POINT (-4.50770 48.39803),Université de Bretagne Occidentale,Université,"48.39803,-4.5077",19 986
247,POINT (6.17634 48.69620),Université de Lorraine,Grand établissement,"48.6962,6.17634",58 925


In [51]:
df_schools = gpd.GeoDataFrame(df_schools)
gpd.sjoin(df_schools, gdf2, how='left', op='within')
#au vu du résultat, peut-être y a-t-il un problème dans les coordonnées ? 

/opt/mamba/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3373: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,geometry,uo_lib,type_d_etablissement,coordonnees,inscrits_2017,index_right,title,Longitude,Latitude,LIBGEO
0,POINT (-3.73045 40.44138),Casa de Velázquez de Madrid,Autre établissement,"40.44138,-3.73045",nd,8.0,Taur,1.442990,43.607800,Toulouse
0,POINT (-3.73045 40.44138),Casa de Velázquez de Madrid,Autre établissement,"40.44138,-3.73045",nd,9.0,Les Humanités,1.403980,43.578300,Toulouse
0,POINT (-3.73045 40.44138),Casa de Velázquez de Madrid,Autre établissement,"40.44138,-3.73045",nd,10.0,Colonel Roche,1.474110,43.563999,Toulouse
0,POINT (-3.73045 40.44138),Casa de Velázquez de Madrid,Autre établissement,"40.44138,-3.73045",nd,11.0,Nobel,2.138980,43.912498,Albi
0,POINT (-3.73045 40.44138),Casa de Velázquez de Madrid,Autre établissement,"40.44138,-3.73045",nd,12.0,Jacqueline Auriol,1.476040,43.556000,Ramonville-Saint-Agne
...,...,...,...,...,...,...,...,...,...,...
248,POINT (0.12896 49.49647),Université Le Havre Normandie,Université,"49.49647,0.12896",7 880,783.0,Cité Grémillon (secteur d'Hérouville-Saint-Clair),-0.339214,49.203205,Hérouville-Saint-Clair
248,POINT (0.12896 49.49647),Université Le Havre Normandie,Université,"49.49647,0.12896",7 880,784.0,Résidence Antide Janvier,6.015343,47.236507,Besançon
248,POINT (0.12896 49.49647),Université Le Havre Normandie,Université,"49.49647,0.12896",7 880,785.0,Résidence Jean Paul Sartre,6.853714,47.638988,Belfort
248,POINT (0.12896 49.49647),Université Le Havre Normandie,Université,"49.49647,0.12896",7 880,786.0,Maret,5.035332,47.322498,Dijon


In [50]:
df_schools = gpd.GeoDataFrame(df_schools)
df_schools.sjoin_nearest(gdf2, how='left', distance_col='Distances')

,geometry,uo_lib,type_d_etablissement,coordonnees,inscrits_2017,index_right,title,Longitude,Latitude,LIBGEO,Distances
0,POINT (-3.73045 40.44138),Casa de Velázquez de Madrid,Autre établissement,"40.44138,-3.73045",nd,691,Boutonnet (Cité universitaire),3.871326,43.623913,Montpellier,0.0
0,POINT (-3.73045 40.44138),Casa de Velázquez de Madrid,Autre établissement,"40.44138,-3.73045",nd,521,Triolet (Cité universitaire),3.860768,43.629398,Montpellier,0.0
0,POINT (-3.73045 40.44138),Casa de Velázquez de Madrid,Autre établissement,"40.44138,-3.73045",nd,415,Triolet II (Résidence universitaire),3.860738,43.629593,Montpellier,0.0
0,POINT (-3.73045 40.44138),Casa de Velázquez de Madrid,Autre établissement,"40.44138,-3.73045",nd,109,Triolet I (Résidence universitaire),3.860738,43.629593,Montpellier,0.0
0,POINT (-3.73045 40.44138),Casa de Velázquez de Madrid,Autre établissement,"40.44138,-3.73045",nd,693,Colombière 2 (Résidence universitaire),3.853663,43.632425,Montpellier,0.0
...,...,...,...,...,...,...,...,...,...,...,...
248,POINT (0.12896 49.49647),Université Le Havre Normandie,Université,"49.49647,0.12896",7 880,175,Beaune,5.076746,47.314262,Dijon,0.0
248,POINT (0.12896 49.49647),Université Le Havre Normandie,Université,"49.49647,0.12896",7 880,658,IRTESS,5.069096,47.318577,Dijon,0.0
248,POINT (0.12896 49.49647),Université Le Havre Normandie,Université,"49.49647,0.12896",7 880,786,Maret,5.035332,47.322498,Dijon,0.0
248,POINT (0.12896 49.49647),Université Le Havre Normandie,Université,"49.49647,0.12896",7 880,695,Jean Monnet,5.756447,49.524319,Longwy,0.0


In [52]:
join_right_df = df_schools.sjoin(gdf2, how="right")
join_right_df

,index_left,uo_lib,type_d_etablissement,coordonnees,inscrits_2017,title,Longitude,Latitude,LIBGEO,geometry
0,1,École supérieure des sciences économiques et c...,École,"49.03397,2.078427",6 103,Résidence Vauban,2.044151,48.779652,Guyancourt,"POLYGON ((12.04415 48.77965, 11.99600 47.79948..."
0,2,École supérieure d'ingénieurs des travaux de l...,École,"49.10042,6.21616",237,Résidence Vauban,2.044151,48.779652,Guyancourt,"POLYGON ((12.04415 48.77965, 11.99600 47.79948..."
0,3,École supérieure d'ingénieurs Léonard-de-Vinci,École,"48.8962,2.2359",2 202,Résidence Vauban,2.044151,48.779652,Guyancourt,"POLYGON ((12.04415 48.77965, 11.99600 47.79948..."
0,4,Université Claude Bernard - Lyon 1,Université,"45.77962,4.86518",38 002,Résidence Vauban,2.044151,48.779652,Guyancourt,"POLYGON ((12.04415 48.77965, 11.99600 47.79948..."
0,5,École nationale supérieure d'ingénieurs de Caen,École,"49.21467,-0.36735",705,Résidence Vauban,2.044151,48.779652,Guyancourt,"POLYGON ((12.04415 48.77965, 11.99600 47.79948..."
...,...,...,...,...,...,...,...,...,...,...
787,244,Institut supérieur d’agriculture Rhône-Alpes,École,"45.727895,4.836377",937,Antipodes,5.072239,47.310177,Dijon,"POLYGON ((15.07224 47.31018, 15.02409 46.33001..."
787,245,La Rochelle Université,Université,"46.14774,-1.15504",8 373,Antipodes,5.072239,47.310177,Dijon,"POLYGON ((15.07224 47.31018, 15.02409 46.33001..."
787,246,Université de Bretagne Occidentale,Université,"48.39803,-4.5077",19 986,Antipodes,5.072239,47.310177,Dijon,"POLYGON ((15.07224 47.31018, 15.02409 46.33001..."
787,247,Université de Lorraine,Grand établissement,"48.6962,6.17634",58 925,Antipodes,5.072239,47.310177,Dijon,"POLYGON ((15.07224 47.31018, 15.02409 46.33001..."


In [53]:
gpd.show_versions() 


SYSTEM INFO
-----------
python     : 3.10.4 | packaged by conda-forge | (main, Mar 24 2022, 17:38:57) [GCC 10.3.0]
executable : /opt/mamba/bin/python
machine    : Linux-5.10.0-16-amd64-x86_64-with-glibc2.31

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : 3.11.1
GEOS lib   : /opt/mamba/lib/libgeos_c.so
GDAL       : 3.5.3
GDAL data dir: /opt/mamba/share/gdal
PROJ       : 9.1.0
PROJ data dir: /opt/mamba/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.12.1
numpy      : 1.23.5
pandas     : 1.5.1
pyproj     : 3.4.0
shapely    : 1.8.5.post1
fiona      : 1.8.22
geoalchemy2: None
geopy      : 2.3.0
matplotlib : 3.6.2
mapclassify: 2.4.3
pygeos     : 0.13
pyogrio    : None
psycopg2   : None
pyarrow    : 9.0.0
rtree      : 1.0.1


In [54]:
#gdf_private['geometry'] = gdf_private['geometry'].apply(wkt.loads) #on transforme en format wkt (sinon ça ne marche pas)
#b = gpd.GeoDataFrame()  #on transforme en geodataframe 
gdf.set_geometry("geometry")
df_schools.set_geometry("geometry")
df_s =  gpd.GeoDataFrame(df_schools) 

### Autre essai

In [56]:
df_schools = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/8f040d23-a09f-4dee-aa19-0892602490d8", sep = ';')

In [57]:
df_schools = df_schools[(df_schools["reg_id"] != "R00") &
                       (df_schools["reg_id"] != "R01") &
                       (df_schools["reg_id"] != "R02")  &
                       (df_schools["reg_id"] != "R03") &
                       (df_schools["reg_id"] != "R04") &
                       (df_schools["reg_id"] != "R05") &
                       (df_schools["reg_id"] != "R06")
                       &
                       (df_schools["reg_id"] != "R99")]

In [58]:
df_schools = df_schools.dropna(subset=['coordonnees'])

In [59]:
df_schools = df_schools.reset_index()
df_schools = df_schools[["uo_lib","type_d_etablissement","coordonnees","inscrits_2017"]]

In [60]:
"""
On récupère les donnés géographiques stockées dans la colonne geocalisation 
qui présente d'abord la latitude puis la longitude. 
"""

Longitude = []
Latitude = []
for j in range(0,df_schools.shape[0]):
    geocalisation = df_schools["coordonnees"][j]
    geocalisation = geocalisation.replace(',', ' ')
    word =  nltk.word_tokenize(geocalisation, language='french')
    Latitude.append(float(word[0]))
    Longitude.append(float(word[1]))
df_schools['Longitude'] = Longitude
df_schools['Latitude'] = Latitude

In [61]:
df_schools.drop(['coordonnees'], axis = 1, inplace = True)
df_schools.columns = ['Name','type','student_number', 'Longitude', 'Latitude']

In [62]:
df_schools

,Name,type,student_number,Longitude,Latitude
0,École supérieure des sciences économiques et c...,École,6 103,2.078427,49.033970
1,École supérieure d'ingénieurs des travaux de l...,École,237,6.216160,49.100420
2,École supérieure d'ingénieurs Léonard-de-Vinci,École,2 202,2.235900,48.896200
3,Université Claude Bernard - Lyon 1,Université,38 002,4.865180,45.779620
4,École nationale supérieure d'ingénieurs de Caen,École,705,-0.367350,49.214670
...,...,...,...,...,...
234,Institut supérieur d’agriculture Rhône-Alpes,École,937,4.836377,45.727895
235,La Rochelle Université,Université,8 373,-1.155040,46.147740
236,Université de Bretagne Occidentale,Université,19 986,-4.507700,48.398030
237,Université de Lorraine,Grand établissement,58 925,6.176340,48.696200


## Fonction distance

In [63]:
!pip install geopy
from geopy.distance import geodesic as GD

def distance (coord1, coord2):
    print(GD(coord1, coord2).km)

In [64]:
A=[]
a=0
for i in range (0,df_schools.shape[0]):
    a=(df_schools['Latitude'][i],df_schools['Longitude'][i])
    A.append(a)
df_schools['coordonnees']=A

In [65]:
print (df_schools)

                                                  Name                 type  \
0    École supérieure des sciences économiques et c...                École   
1    École supérieure d'ingénieurs des travaux de l...                École   
2       École supérieure d'ingénieurs Léonard-de-Vinci                École   
3                   Université Claude Bernard - Lyon 1           Université   
4      École nationale supérieure d'ingénieurs de Caen                École   
..                                                 ...                  ...   
234       Institut supérieur d’agriculture Rhône-Alpes                École   
235                             La Rochelle Université           Université   
236                 Université de Bretagne Occidentale           Université   
237                             Université de Lorraine  Grand établissement   
238                      Université Le Havre Normandie           Université   

    student_number  Longitude   Latitude           

In [66]:
df_schools.shape[0]
df.shape[0]

788

In [67]:
coord1 = (gdf['Latitude'][1],gdf['Longitude'][1])
coord2 = (df_schools['Latitude'][1],df_schools['Longitude'][1])

In [68]:
a = df_s['coordonnees'][1].split(',')
print(tuple(a))

('49.03397', '2.078427')


In [69]:
a[1]

'2.078427'